In [1]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.select import Select
import plotly.express as px

In [11]:
wd = webdriver.Safari()

In [3]:
cd ~/Downloads/

/Users/rodriguesra/Dropbox/My Mac (Rafael’s MacBook Pro)/Downloads


In [4]:
def busca_carteira_teorica(indice, espera=3):
    url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'
    wd.get(url)
    sleep(espera)
    
    dropdown_menu = wd.find_element_by_id('segment')
    Select(dropdown_menu).select_by_visible_text('Setor de Atuação')
    sleep(espera)

    wd.find_element_by_link_text('Download').click()
    sleep(espera)
    
    arquivos = !ls -1t *.csv
    columns = ['Setor', 'Código', 'Ação', 'Tipo', 'Qtde. Teórica', 'Part. (%)', 'Part. (%)Acum.', 'Helper_col']
    
    df = pd.read_csv(arquivos[0], sep=';', encoding='latin-1', thousands='.', decimal=',', skiprows=2, 
                     skipfooter=2, engine='python')
    df.columns = columns
    df.drop(columns='Helper_col', axis=1, inplace=True)
    return df

In [15]:
df = busca_carteira_teorica('smll')

In [16]:
df['Sub-Setor'] = df['Setor'].apply(lambda s: s[s.rfind('/') + 1:].strip())
df['Setor'] = df['Setor'].apply(lambda s: s[:s.rfind('/')].strip())

In [7]:
def conserta_setores(setor):
    if setor == 'Cons N  Básico' or setor == 'Cons N Cíclico': return 'Consumo Não-Cíclico'
    if setor == 'Financeiro e Outros' or setor == 'Financ e Outros': return 'Financeiro'
    if setor == 'Telecomunicaçã': return 'Telecomunicações'
    if setor == 'Tec.Informação': return 'Tec. Informação'
    if setor == 'Utilidade Públ': return 'Utilidade Pública'
    else: return setor

In [8]:
df['Setor'] = df['Setor'].apply(conserta_setores)

In [17]:
fig = px.sunburst(data_frame=df, path=['Setor', 'Sub-Setor', 'Código'], values='Part. (%)', height=900)
fig.update_traces(textfont_color='white', textfont_size=14, hovertemplate='<b>%{label}: </b> %{value:.2f}%')
fig.show()

In [ ]:
fig = px.treemap(data_frame=df, path=['Setor', 'Sub-Setor', 'Código'], values='Part. (%)', height=900)
fig.update_traces(textfont_color='white', textfont_size=14, hovertemplate='<b>%{label}: </b> %{value:.2f}%')
fig.show()

In [ ]:
!